In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L1\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['MIDATL.csv', 'SOUTH.csv', 'WEST.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[0]))

,Net,Temperature
0,31660.636,-7.6
1,30766.066,-8.4
2,30201.596,-8.3
3,29901.406,-6.8
4,30084.427,-7.0
...,...,...
43819,31844.076,4.8
43820,30687.078,4.4
43821,29583.295,3.7
43822,28458.691,2.8


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[0]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_MIDATL,Temp_MIDATL
0,31660.636,-7.6
1,30766.066,-8.4
2,30201.596,-8.3
3,29901.406,-6.8
4,30084.427,-7.0
...,...,...
43819,31844.076,4.8
43820,30687.078,4.4
43821,29583.295,3.7
43822,28458.691,2.8


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_MIDATL,Temp_MIDATL,Lag-2,Lag-7
0,38996.785,-11.3,32288.116,31660.636
1,38439.636,-12.9,31695.589,30766.066
2,38168.687,-13.8,31377.350,30201.596
3,38288.055,-14.5,31461.963,29901.406
4,39122.047,-15.0,32349.539,30084.427
...,...,...,...,...
43651,31844.076,4.8,31492.518,31223.944
43652,30687.078,4.4,30943.798,30934.643
43653,29583.295,3.7,29728.352,30619.720
43654,28458.691,2.8,28090.471,29955.427


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 13s 8ms/step - loss: 0.0137 - mape: 7115.8530 - mae: 0.0902 - val_loss: 0.0088 - val_mape: 49.1574 - val_mae: 0.0741
Epoch 2/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0090 - mape: 6271.5713 - mae: 0.0741 - val_loss: 0.0110 - val_mape: 47.7333 - val_mae: 0.0795
Epoch 3/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0068 - mape: 5672.6392 - mae: 0.0639 - val_loss: 0.0141 - val_mape: 58.4397 - val_mae: 0.0953
Epoch 4/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0062 - mape: 5234.6899 - mae: 0.0608 - val_loss: 0.0104 - val_mape: 48.8504 - val_mae: 0.0775
Epoch 5/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0057 - mape: 5209.0024 - mae: 0.0580 - val_loss: 0.0126 - val_mape: 32.3401 - val_mae: 0.0811
Epoch 6/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0051 - mape: 5514.5122 - mae: 0.0545 - val_loss: 0.0104

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0025 - mape: 2735.2952 - mae: 0.0373 - val_loss: 0.0090 - val_mape: 34.8220 - val_mae: 0.0664
Epoch 49/100
1105/1105 [==============================] - 10s 9ms/step - loss: 0.0024 - mape: 2433.3291 - mae: 0.0370 - val_loss: 0.0086 - val_mape: 34.7846 - val_mae: 0.0660
Epoch 50/100
1105/1105 [==============================] - 11s 10ms/step - loss: 0.0024 - mape: 2596.8398 - mae: 0.0365 - val_loss: 0.0104 - val_mape: 30.8119 - val_mae: 0.0711
Epoch 51/100
1105/1105 [==============================] - 10s 9ms/step - loss: 0.0023 - mape: 2311.4194 - mae: 0.0361 - val_loss: 0.0091 - val_mape: 33.1080 - val_mae: 0.0668
Epoch 52/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0023 - mape: 2214.0381 - mae: 0.0360 - val_loss: 0.0091 - val_mape: 35.6392 - val_mae: 0.0672
Epoch 53/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0023 - mape: 2571.2671 - mae: 0.0357 - val_loss: 0.0102 - v

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0012 - mape: 1408.7760 - mae: 0.0266 - val_loss: 0.0091 - val_mape: 39.0452 - val_mae: 0.0676
Epoch 96/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0012 - mape: 1354.5393 - mae: 0.0264 - val_loss: 0.0092 - val_mape: 37.4003 - val_mae: 0.0678
Epoch 97/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0012 - mape: 1350.4897 - mae: 0.0266 - val_loss: 0.0101 - val_mape: 37.6298 - val_mae: 0.0715
Epoch 98/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0012 - mape: 1607.4589 - mae: 0.0259 - val_loss: 0.0099 - val_mape: 37.6644 - val_mae: 0.0705
Epoch 99/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0012 - mape: 1546.8667 - mae: 0.0258 - val_loss: 0.0090 - val_mape: 38.8431 - val_mae: 0.0674
Epoch 100/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0012 - mape: 1503.2905 - mae: 0.0258 - val_loss: 0.0097 - val_

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 812.6091666221619


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 6ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.2799341 , 0.23662221, 0.21669614, ..., 0.42282987, 0.38114357,
       0.33922747], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.44718243, 0.40340449, 0.37590709, ..., 0.38274799, 0.35681528,
       0.33305307])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,32377.575,25124.642578
1,30479.090,23246.369141
2,29286.631,22382.250000
3,28585.638,22154.300781
4,28604.834,22210.740234
...,...,...
4363,31844.076,32681.595703
4364,30687.078,32368.062500
4365,29583.295,31321.496094
4366,28458.691,29513.716797


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

8.56913053385808